In [1]:
from time import time

def time_elapsed(t):
    seconds = time() - t
    
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    
    return "{:.0f}h {:.0f}m {:f}s".format(hours, minutes, seconds)

# Project Euler problem 186 - Connectedness of a network

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=186)

## Description

Here are the records from a busy telephone system with one million users:

| RecNr | Caller | Called |
|:-----:|:------:|:------:|
| 1     | 200007 | 100053 |
| 2     | 600183 | 500439 |
| 3     | 600863 | 701497 |
| ...   | ...    | ...    |

The telephone number of the caller and the called number in record $n$ are $\mathrm{Caller}(n) = S_{2n-1}$ and $\mathrm{Called}(n) = S_{2n}$ where $S_{1,2,3,\ldots}$ come from the "Lagged Fibonacci Generator":

$$
\begin{eqnarray}
S_k &=& [100003 - 200003k + 300007k^3] \mod 1000000 &,& 1 \leq k \leq 55 \\
S_k &=& [S_{k-24} + S_{k-55}] \mod 1000000 &,& 56 \leq k
\end{eqnarray}
$$

If $\mathrm{Caller}(n) = \mathrm{Called}(n)$ then the user is assumed to have misdialled and the call fails; otherwise the call is successful.

From the start of the records, we say that any pair of users X and Y are friends if X calls Y or vice-versa. Similarly, X is a friend of a friend of Z if X is a friend of Y and Y is a friend of Z; and so on for longer chains.

The Prime Minister's phone number is 524287. After how many successful calls, not counting misdials, will 99% of the users (including the PM) be a friend, or a friend of a friend etc., of the Prime Minister?

In [3]:
from functools import lru_cache

@lru_cache(maxsize=2**8)
def S(k, mod=10**6):
    if k >= 56:
        return (S(k-24) + S(k-55)) % mod
    else:
        return (100003 - 200003*k + 300007*k**3) % mod

print(S(1))
print(S(2))
print(S(3))

200007
100053
600183


In [4]:
from itertools import count

def call_generator(mod=10**6):
    for n in count(1):
        caller = S(2*n-1, mod=mod)
        called = S(2*n, mod=mod)
        if caller != called:
            yield caller, called

gen = call_generator()

print(next(gen))
print(next(gen))
print(next(gen))

(200007, 100053)
(600183, 500439)
(600863, 701497)


In [5]:
class Node:
    """Node in disjoint set data structure"""

    def __init__(self, id):

        self.id = id
        self.parent = self
        self.rank = 0
        self.size = 1

    def find(self):
        """find by path halving"""
        x = self
        while x.parent != x:
            x.parent = x.parent.parent
            x = x.parent
        return x

def union(x, y):
    x_root = x.find()
    y_root = y.find()

    # x and y are already in the same set
    if x_root == y_root:
        return

    # x and y are not in same set, so we merge them
    if x_root.size < y_root.size:
        x_root, y_root = y_root, x_root # swap xRoot and yRoot

    # merge yRoot into xRoot
    y_root.parent = x_root
    x_root.size = x_root.size + y_root.size

In [10]:
limit = 10**6
pm_id = 524287

t0 = time()

goal = limit - limit//100

callers = [Node(i) for i in range(limit)]

cg = call_generator()

ncalls = 0
pm_size = 1
while pm_size < goal:
    caller_id, called_id = next(cg)
    ncalls += 1
    
    union(callers[caller_id], callers[called_id])
    
    pm_size = callers[pm_id].find().size

print("Result: {}".format(ncalls))
print("Time elapsed = {}".format(time_elapsed(t0)))

Result: 2325629
Time elapsed = 0h 0m 12.654385s
